# Checkpoint 5

## GRADER: Update the current_directory below to whichever directory you are loading this file from to be able to load the proper CSV files.

In [11]:
import csv
import os
from numpy import genfromtxt
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score

"""
GRADER: Update the current_directory below to whichever directory 
you are loading this file from to be able to load the proper CSV files.

This should be the directory to get to checkpoint_5
"""

# Update HERE
current_directory = '/.../checkpoint_5/src'

os.chdir(current_directory)
os.getcwd()

'/Users/ilanponsky/Desktop/checkpoint_5/src'

In [12]:
# Load CSV files (data and labels)
data = pd.read_csv("cpdb_public_checkpoint5.csv") 
data = data.dropna()

officer_ranking_list = pd.read_csv('cpdb_public_officer_rank_list.csv')

In [13]:
data_np = np.array(data)
officer_ranking_list = np.array(officer_ranking_list)
or_list = []

for rank in officer_ranking_list:
    or_list.append(rank[0])

## Mappings to "Numerify" the Data

In [14]:
victim_race_labs = [
    (0, 'Asian/Pacific Islander'),
    (1, 'Hispanic'),
    (2, 'White'),
    (3, 'Native American/Alaskan Native'),
    (4, 'Black')
]

officer_race_labs = [
    (0, 'Asian/Pacific'),
    (1, 'Hispanic'),
    (2, 'White'),
    (3, 'Native American/Alaskan Native'),
    (4, 'Black')
]

or_list = []
for i, rank in enumerate(officer_ranking_list):
    or_list.append((i, rank[0]))
    
or_list.append((16, 'Assistant Deputy Superintendent'))
print(or_list)

[(0, 'Lieutenant'), (1, 'Assistant Superintendent'), (2, 'Sergeant'), (3, 'Commander'), (4, 'Police Officer'), (5, 'Detective'), (6, 'Superintendent Of Police'), (7, 'Other'), (8, 'Deputy Chief'), (9, 'Director Of Caps'), (10, 'Chief'), (11, "Superintendent'S Chief Of Staff"), (12, 'Captain'), (13, 'Field Training Officer'), (14, 'Deputy Superintendent'), (15, 'First Deputy Superintendent'), (16, 'Assistant Deputy Superintendent')]


## Below is a function to Numerify data to convert all data to float values

In [39]:
# Changing Strings to numerical values based on mapping above

def numerify(data):
    new_data = []
    for i, row in enumerate(data):
        temp = row
        
        # Victim Race
        race = temp[2]
        for j in victim_race_labs:
            if race == j[1]:
                temp[2] = j[0]
        
        # Officer Race
        o_race = temp[4]
        for j in officer_race_labs:
            if o_race == j[1]:
                temp[4] = j[0]
        
        # Officer Rank
        rank = temp[5]
        for j in or_list:
            if rank == j[1]:
                temp[5] = j[0]
                
        temp = temp.tolist()
        new_data.append(temp)
    
    new_data = np.array(new_data, dtype=float)
    new_data = new_data[:, 1:]
    return new_data

In [45]:
# Run Numerify
num_data = numerify(data_np)

## Below are the machine learning models we ran on the data

### Question 1 analysis using an SVM model

In [46]:
# Full feature set, targets are counts of allegations per officer

features = num_data[:, 0:-1]
targets = num_data[:, -1]

In [47]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2)

# Run Model
model = SVC()
model.fit(X_train, y_train)

predictions1 = model.predict(X_test)

In [48]:
print('Number of allegations predictions: \n', predictions1[0:20])

Number of allegations predictions: 
 [2. 3. 4. 6. 1. 3. 6. 4. 1. 2. 2. 2. 2. 1. 6. 2. 2. 2. 6. 2.]


In [49]:
print('Results for predicting amount of allegations per officer:')
print('---------------------------------------------------------')
f1 = f1_score(predictions1, y_test, average='weighted')
print('F1 score:\t', f1)

accuracy = accuracy_score(predictions1, y_test)
print('Accuracy Score: ', accuracy)

Results for predicting amount of allegations per officer:
---------------------------------------------------------
F1 score:	 0.9525559658566799
Accuracy Score:  0.9365466748017084


### Question 2 analysis using an SVM model

In [21]:
# Full feature set, targets are most commonly targeted racial group per officer

features = np.delete(num_data, 1, axis=1)
targets = num_data[:, 1]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2)

# Run Model
model = SVC()
model.fit(X_train, y_train)

predictions2 = model.predict(X_test)

In [22]:
print('Label for most commonly targeted racial group predictions: \n', predictions2[0:20])

Label for most commonly targeted racial group predictions: 
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]


In [23]:
print('Results for predicting most commonly targeted racial group per officer:')
print('-----------------------------------------------------------------------')
f1 = f1_score(predictions2, y_test, average='weighted')
print('F1 score:\t', f1)

accuracy = accuracy_score(predictions2, y_test)
print('Accuracy Score: ', accuracy)

Results for predicting most commonly targeted racial group per officer:
-----------------------------------------------------------------------
F1 score:	 0.7750373692077728
Accuracy Score:  0.6327028676021964
